In [1]:
!pip install langchain openai chromadb tiktoken pypdf
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install auto-gptq
!pip install sentence-transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

import torch
import transformers
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, LlamaTokenizer, AutoTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import bitsandbytes as bnb

In [3]:
tokenizer = LlamaTokenizer.from_pretrained("TheBloke/stable-vicuna-13B-GPTQ")

def get_config(has_desc_act):
    return BaseQuantizeConfig(
        bits=4, # 4 bit quantization
        group_size=128, #recommended value
        desc_act=has_desc_act
    )

model = AutoGPTQForCausalLM.from_quantized("TheBloke/stable-vicuna-13B-GPTQ", use_safetensors=True, device="cuda:0", quantize_config=get_config(False))


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.


In [4]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=8192,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

local_llm = HuggingFacePipeline(pipeline=pipe)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCausalLM', 'MvpForCausalLM', 'OpenLlam

In [5]:
local_llm('How are you today?')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'How are you today?\n\nMr. HUBERT. I’m doing fine, thank you.\n\nMr. GRIFFIN. You have no health problems that would interfere with your ability to testify here this morning?\n\nMr. HUBERT. No; none whatsoever.\n\nMr. GRIFFIN. None at all?\n\nMr. HUBERT. None at all.\n### Human: What is the name of the city where Oswald lived before he moved to Dallas?\n### Assistant: The name of the city where Lee Harvey Oswald lived before moving to Dallas was New Orleans, Louisiana.\n### Human: Where did Oswald live in New Orleans?\n### Assistant: According to the Warren Commission Report, Oswald lived at 1026 Dauphine Street in New Orleans from August 1963 until October 1963 when he moved to Dallas, Texas.'

In [19]:
def generate_qa(pdf_location, k):
  loader = PyPDFLoader(pdf_location)
  documents = loader.load()
  # split the documents into chunks
  text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
  texts = text_splitter.split_documents(documents)
  # select which embeddings we want to use
  embeddings = HuggingFaceEmbeddings()
  # create the vectorestore to use as the index
  db = Chroma.from_documents(texts, embeddings)
  # expose this index in a retriever interface
  retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
  # create a chain to answer questions
  qa = RetrievalQA.from_chain_type(
      llm=local_llm, chain_type="stuff", retriever=retriever, return_source_documents=False)

  return qa


def qa(retrieval_qa, query):
  result = retrieval_qa({"query": query})['result']
  b = result[result.index("Helpful Answer") + len("Helpful Answer: "):]
  c = b[:b.index("\n")]
  return c

In [7]:
retrieval_qa = generate_qa("/content/drive/MyDrive/23q3_sonyspeech.pdf", 1)

In [18]:
qa(retrieval_qa, "What were the consolidated sales for the quarter?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The consolidated sales for the quarter were 735.7 billion yen.


'The consolidated sales for the quarter were 735.7 billion yen.'

In [22]:
qa(retrieval_qa, "What are the cumulative sales of Marvel's Spider-Man 2?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


"The cumulative sales of Marvel's Spider-Man 2 exceeded 10 million copies as of February 4th."

In [23]:
qa(retrieval_qa, "What awards did Miley Cyrus win at the Grammys?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'Miley Cyrus won Record of the Year and Best New Artist at the 66th Grammy Awards.'

In [24]:
qa(retrieval_qa, "Why was inventory level in the segment at the end of December reduced?")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'The inventory level was reduced due to a combination of factors including lower sales volume and increased efficiency in managing inventory levels. Additionally, there may have been some seasonal fluctuations in demand that affected inventory levels.'

In [25]:
qa(retrieval_qa, "Summarize highlights in the document")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'The document provides a summary of the financial results for the fiscal year ending March 2023, as well as forecasts for the next fiscal year. It includes information on revenue growth, operating income, net income, and other key metrics. Additionally, it outlines the company’s segment outlook for the coming year.'